<a href="https://colab.research.google.com/github/ufofon2/catalog-service/blob/main/03_pinecone_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 주요 패키지 설치


In [ ]:
%pip install -qU python-dotenv

In [ ]:
%pip install -qU langchain langchain-community langchain-openai langchain-pinecone

In [ ]:
%pip install -qU pinecone-client

In [ ]:
%pip install -qU pandas matplotlib tqdm ipywidgets

### 환경변수 설정


In [ ]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수를 로드합니다.
load_dotenv()

# 환경 변수에서 PINECONE_API_KEY를 가져옵니다.
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

### Pinecone 클라이언트 초기화


In [ ]:
from pinecone import Pinecone, ServerlessSpec

# Pinecone 클라이언트를 초기화합니다.
# PINECONE_API_KEY는 환경 변수에서 가져온 API 키입니다.
pc = Pinecone(api_key=PINECONE_API_KEY)

### 레코드 업서트(Upsert)


In [ ]:
# "embedding-3d"라는 이름의 인덱스를 초기화합니다.
index = pc.Index("embedding-3d")

# 인덱스의 통계 정보를 설명하는 메서드를 호출하고, 그 결과를 출력합니다.
# 인덱스의 통계 정보에는 인덱스에 저장된 벡터의 수, 차원, 메타데이터 등의 정보가 포함될 수 있습니다.
index.describe_index_stats()

In [ ]:
# 벡터를 인덱스에 업서트(upsert)합니다.
# "embedding-3d-ns1" 네임스페이스에 벡터를 추가합니다.
index.upsert(
    vectors=[
        {
            "id": "vec1",
            "values": [1.0, 1.5, 2.0],
            "metadata": {"genre": "drama"}
        }, {
            "id": "vec2",
            "values": [2.0, 1.0, 0.5],
            "metadata": {"genre": "action"}
        }, {
            "id": "vec3",
            "values": [0.1, 0.3, 0.5],
            "metadata": {"genre": "drama"}
        }, {
            "id": "vec4",
            "values": [1.0, 2.5, 3.5],
            "metadata": {"genre": "action"}
        }, {
            "id": "vec5",
            "values": [3.0, 1.2, 1.3],
            "metadata": {"genre": "action"}
        }, {
            "id": "vec6",
            "values": [0.3, 1.1, 2.5],
            "metadata": {"genre": "drama"}
        }
    ],
    namespace="embedding-3d-ns1"
)

### 쿼리하기


In [ ]:
# 주어진 벡터와 유사한 상위 k개의 항목을 쿼리하여 응답을 반환합니다.
# namespace (str): 쿼리할 네임스페이스 이름.
# vector (list): 쿼리에 사용할 벡터.
# top_k (int): 반환할 상위 k개의 항목 수.
# include_values (bool): 응답에 벡터 값을 포함할지 여부.
# include_metadata (bool): 응답에 메타데이터를 포함할지 여부.
response = index.query(
    namespace="embedding-3d-ns1",
    vector=[0.1, 0.3, 0.7],
    top_k=3,
    include_values=True,
    include_metadata=True
)

print(response)

In [ ]:
# 주어진 벡터와 유사한 상위 k개의 항목을 필터 조건으로 쿼리하여 응답을 반환합니다.
# filter (dict): 쿼리할 벡터의 필터 조건.
response = index.query(
    namespace="embedding-3d-ns1",
    vector=[0.1, 0.3, 0.7],
    top_k=3,
    include_values=True,
    include_metadata=True,
    filter={
        "genre": {"$eq": "drama"}
    }
)

print(response)

In [ ]:
# embedding-3d-ns1 네임스페이스의 모든 레코드 id를 조회합니다.
for ids in index.list(namespace="embedding-3d-ns1"):
    print(ids)

In [ ]:
# 지정된 네임스페이스(embedding-3d-ns1)에 있는 모든 ID를 리스트 형태로 반환합니다.
for ids in index.list(namespace="embedding-3d-ns1"):
    # 주어진 ID와 네임스페이스에 해당하는 벡터 데이터를 가져옵니다.
    # 반환된 데이터는 딕셔너리 형태이며, 그 중에서 'vectors' 키에 해당하는 값을 vectors 변수에 저장합니다.
    vectors = index.fetch(ids, namespace="embedding-3d-ns1")['vectors']

print(vectors.values())

In [ ]:
# ids: 벡터 데이터에서 추출한 ID 리스트.
# values: 벡터 데이터에서 추출한 값 리스트.
ids = [v['id'] for v in vectors.values()]
values = [v['values'] for v in vectors.values()]

ids += ["qv"]
values += [[0.1, 0.3, 0.7]]

print(ids)
print(values)

In [ ]:
import matplotlib.pyplot as plt

# 3D 플롯을 생성합니다.
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# 각 벡터의 ID와 값을 사용하여 3D 공간에 점을 그립니다.
for i, id in enumerate(ids):
    if id == "qv":
        # 쿼리 벡터는 노란색 사각형으로 표시합니다.
        ax.scatter(values[i][0], values[i][1], values[i][2], label=id, color='y', s=50, marker='s')
        ax.text(values[i][0] + 0.1, values[i][1] + 0.1, values[i][2] + 0.1, id)
    else:
        # 나머지 벡터는 기본 색상으로 표시합니다.
        ax.scatter(values[i][0], values[i][1], values[i][2], label=id)
        ax.text(values[i][0] + 0.1, values[i][1] + 0.1, values[i][2] + 0.1, id)

# 축 레이블과 제목을 설정합니다.
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Embedding Vector Space')

# 플롯을 표시합니다.
plt.show()

### 서버리스 인덱스 생성


In [ ]:
index_name = "quickstart"

# Pinecone에 있는 모든 인덱스를 순회합니다.
for idx in pc.list_indexes():
    # 인덱스 이름이 "quickstart"와 일치하는 경우 해당 인덱스를 삭제합니다.
    if idx.name == index_name:
        pc.delete_index(idx.name)

In [ ]:
# Pinecone 인덱스를 생성합니다.
# 인덱스 이름은 "quickstart"이고, 차원은 1024, 메트릭은 코사인 유사도를 사용합니다.
# 인덱스는 AWS의 us-east-1 리전에서 서버리스 사양으로 생성됩니다.
pc.create_index(
    name=index_name,
    dimension=1024,  # 모델 차원
    metric="cosine",  # 모델 메트릭
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [ ]:
# Pinecone 클라이언트를 사용하여 현재 사용 가능한 모든 인덱스의 목록을 반환합니다.
pc.list_indexes()

### 임베딩 벡터 생성


Pinecone에서 레코드 id는 각 벡터를 고유하게 식별하는 데 사용됩니다.

Pinecone에서 id에 대한 주요 제약사항은 다음과 같습니다:

- 고유성: 각 벡터의 id는 인덱스 내에서 고유해야 합니다.
- 문자열 형식: id는 문자열 형태여야 합니다.
- 길이 제한: id의 최대 길이는 512바이트입니다.
- 허용 문자: id에는 영숫자, -, \_, #, :를 사용할 수 있습니다.
- 대소문자 구분: id는 대소문자를 구분합니다.
- 공백 불가: id에는 공백을 포함할 수 없습니다

이러한 제약사항들은 Pinecone 시스템에서 벡터를 효율적으로 관리하고 검색할 수 있도록 하기 위해 설계되었습니다. id를 생성할 때 이러한 규칙을 준수하면 Pinecone 인덱스와 원활하게 작업할 수 있습니다.


In [ ]:
data = [
    {"id": "vec1", "text": "사과는 달콤하고 아삭한 식감으로 유명한 인기 있는 과일입니다."},
    {"id": "vec2", "text": "애플이라는 기술 회사는 아이폰과 같은 혁신적인 제품으로 유명합니다."},
    {"id": "vec3", "text": "많은 사람들이 건강한 간식으로 사과를 즐겨 먹습니다."},
    {"id": "vec4", "text": "애플 주식회사는 세련된 디자인과 사용자 친화적인 인터페이스로 기술 산업을 혁신했습니다."},
    {"id": "vec5", "text": "하루에 사과 하나면 의사를 멀리할 수 있다는 속담이 있습니다."},
    {"id": "vec6", "text": "애플 컴퓨터 회사는 1976년 4월 1일 스티브 잡스, 스티브 워즈니악, 로널드 웨인에 의해 파트너십으로 설립되었습니다."}
]

multilingual-e5-large 임베딩 모델

- 마이크로소프트가 만든 1024 차원을 가진 고성능의 오픈소스 다국어 임베딩 모델입니다.
- 코사인 메트릭을 사용하고, 입력 토큰으로 최대 507개를, 최대 96번의 배치 처리를 허용합니다.
- 지저분한 데이터에서도 잘 작동하고, 짧은 쿼리에 대해 중간 길이의 텍스트 단락(1-2 단락)을 반환하는 데 적합합니다.


In [ ]:
# 텍스트 데이터를 임베딩 벡터로 변환합니다.
# inputs 매개변수는 임베딩할 텍스트 데이터를 지정합니다. data 리스트의 각 항목에서 'text' 키에 해당하는 값을 추출하여 리스트로 만듭니다.
# parameters 매개변수는 추가적인 파라미터를 지정합니다.
# "input_type"은 입력 데이터의 유형을 지정합니다. "passage"는 입력 데이터가 문단 또는 긴 텍스트 조각임을 나타냅니다.
# 이는 모델이 입력 데이터를 처리하는 방식을 결정하는 데 사용됩니다.
# "truncate"는 입력 데이터가 모델의 최대 길이를 초과할 경우 어떻게 처리할지를 지정합니다. "END"는 입력 데이터가 너무 길 경우 끝부분을
# 잘라내는(truncate) 방식을 의미합니다. 즉, 입력 데이터의 앞부분은 유지하고 뒷부분을 잘라내어 모델의 최대 길이에 맞춥니다.
embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[d['text'] for d in data],
    parameters={"input_type": "passage", "truncate": "END"}
)

# 첫 번째 임베딩 벡터를 출력합니다.
print(embeddings[0])

### 레코드 업서트(Upsert)


In [ ]:
import time

# 인덱스가 준비될 때까지 대기합니다.
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

# 인덱스를 초기화합니다.
index = pc.Index(index_name)

# 벡터 리스트를 초기화합니다.
vectors = []
for d, e in zip(data, embeddings):
    # 각 데이터와 임베딩을 결합하여 벡터를 생성합니다.
    vectors.append({
        "id": d['id'],
        "values": e['values'],
        "metadata": {'text': d['text']}
    })

# 벡터를 인덱스에 업서트(upsert)합니다. quickstart_ns1 네임스페이스에 벡터를 추가합니다.
index.upsert(
    vectors=vectors,
    namespace="quickstart_ns1"
)

### 인덱스 통계 정보 확인


In [ ]:
# Pinecone 인덱스의 통계 정보를 설명하는 메서드를 호출하고, 그 결과를 출력합니다.
# 인덱스의 통계 정보에는 인덱스에 저장된 벡터의 수, 차원, 메타데이터 등의 정보가 포함될 수 있습니다.
print(index.describe_index_stats())

### 쿼리(Query)


#### 쿼리 벡터 생성


In [ ]:
# 쿼리 텍스트를 정의합니다.
query = "애플이라는 기술 회사에 대해 알려주세요."

# 쿼리 텍스트를 임베딩 벡터로 변환합니다.
embedding = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={
        "input_type": "query"  # 입력 타입을 쿼리로 설정합니다.
    }
)

# 첫 번째 임베딩 벡터를 출력합니다.
print(embedding[0])

#### 유사 벡터 검색 쿼리 실행


In [ ]:
# ns1 네임스페이스에서 쿼리 벡터와 유사한 상위 3개의 벡터를 검색합니다.
# 검색 결과에는 벡터 값은 포함되지 않고 메타데이터만 포함됩니다.
results = index.query(
    namespace="quickstart_ns1",
    vector=embedding[0].values,
    top_k=3,
    include_values=False,
    include_metadata=True
)

# 검색 결과를 출력합니다.
print(results)